In [1]:
import numpy as np
from scipy.linalg import sqrtm
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import glob

def load_images(path, num_images):
    images = []
    for filename in glob.glob(f'{path}/*.jpg')[:num_images]:
        img = load_img(filename, target_size=(299, 299))
        img = img_to_array(img)
        img = preprocess_input(img)
        images.append(img)
    return np.array(images)

def calculate_fid(real_images, generated_images):
    model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))
    
    real_features = model.predict(real_images)
    generated_features = model.predict(generated_images)
    
    mu1, sigma1 = real_features.mean(axis=0), np.cov(real_features, rowvar=False)
    mu2, sigma2 = generated_features.mean(axis=0), np.cov(generated_features, rowvar=False)
    
    ssdiff = np.sum((mu1 - mu2)**2.0)
    covmean = sqrtm(sigma1.dot(sigma2))
    
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

def calculate_inception_score(images):
    model = InceptionV3()
    preds = model.predict(images)
    
    scores = []
    for i in range(preds.shape[0]):
        p_yx = preds[i]
        p_y = np.expand_dims(p_yx.mean(axis=0), 0)
        kl_d = p_yx * (np.log(p_yx + 1e-16) - np.log(p_y + 1e-16))
        sum_kl_d = kl_d.sum(axis=1)
        scores.append(np.exp(sum_kl_d))
    
    return np.mean(scores), np.std(scores)

# Example usage
real_images = load_images('path/to/real/images', 1000)
generated_images = load_images('path/to/generated/images', 1000)

fid_score = calculate_fid(real_images, generated_images)
inception_score, inception_std = calculate_inception_score(generated_images)

print(f"FID Score: {fid_score}")
print(f"Inception Score: {inception_score} ± {inception_std}")

ModuleNotFoundError: No module named 'tensorflow'